In [ ]:
! pip install --quiet numpy pandas seaborn matplotlib tqdm
! pip install --quiet datasets "transformers[torch]" scikit-learn

# Rice Leaf Disease Detection - Model Evaluation
This notebook evaluates multiple transformer-based models for rice leaf disease classification.

In [ ]:

import os
import json
import torch
import time
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from tqdm import tqdm
from datasets import load_dataset
from transformers import AutoModelForImageClassification, AutoProcessor
from transformers import ViTHybridForImageClassification, ViTHybridImageProcessor
from sklearn.metrics import classification_report, confusion_matrix
from google.colab import drive


## Function: Model Evaluation
This function loads a pre-trained model and evaluates it on the test dataset.

In [ ]:
def evaluate_model(model_name, dataset, labels, batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu"):
    """Loads a model and evaluates it on the dataset in batches, displaying all failed predictions."""
    print(f"Evaluating {model_name}...")

    # Load model and processor
    if "hybrid" in model_name:
        model = ViTHybridForImageClassification.from_pretrained(model_name).to(device)
        processor = ViTHybridImageProcessor.from_pretrained(model_name)
    else:
        model = AutoModelForImageClassification.from_pretrained(model_name).to(device)
        processor = AutoProcessor.from_pretrained(model_name)

    y_true, y_pred = [], []
    failed_predictions = []

    start_time = time.time()

    for example in tqdm(dataset, desc=f"Testing {model_name}"):
        image, label = example["image"], example["label"]
        inputs = processor(images=image, return_tensors="pt").to(device)

        with torch.no_grad():
            outputs = model(**inputs)
            pred_label = torch.argmax(outputs.logits, dim=-1).cpu().item()

        y_true.append(label)
        y_pred.append(pred_label)

        if pred_label != label:  # Store failed predictions
            failed_predictions.append((image, label, pred_label))

    elapsed_time = time.time() - start_time
    print(f"Model {model_name} evaluation completed in {elapsed_time:.2f} seconds.")
    print(f"Total failed predictions: {len(failed_predictions)}")


    return y_true, y_pred, failed_predictions, elapsed_time


## Function: Generate Report
This function generates and saves a classification report and confusion matrix.

In [ ]:
def generate_report(y_true, y_pred, labels, model_name, output_dir, failed_predictions, elapsed_time):
    """Generates and saves classification report and confusion matrix."""
    model_safe_name = model_name.split("/")[-1]
    model_safe_name = model_safe_name.split("_")[0] + model_safe_name.split("_")[-1]
    output_dir = os.path.join(output_dir, model_name.split("/")[-1])
    os.makedirs(output_dir, exist_ok=True)

    report = classification_report(y_true, y_pred, target_names=labels, output_dict=True)
    cm = confusion_matrix(y_true, y_pred)
    cm_normalized = cm.astype("float") / cm.sum(axis=1, keepdims=True)

    report["evaluation_time_sec"] = elapsed_time

    # Save JSON report
    report_path = os.path.join(output_dir, f"report.json")
    with open(report_path, "w") as f:
        json.dump(report, f, indent=4)

    # Save Excel report
    report_df = pd.DataFrame(report).transpose()
    excel_path = os.path.join(output_dir, f"report.xlsx")

    with pd.ExcelWriter(excel_path) as writer:
        report_df.to_excel(writer, sheet_name="Classification Report")
        pd.DataFrame(cm, index=labels, columns=labels).to_excel(writer, sheet_name="Confusion Matrix")
        pd.DataFrame(cm_normalized, index=labels, columns=labels).to_excel(writer, sheet_name="Normalized Confusion Matrix")

    # Display all failed predictions
    if failed_predictions:
        num_failures = len(failed_predictions)
        cols = 4  # Set columns for visualization
        rows = (num_failures // cols) + (num_failures % cols > 0)

        fig, axes = plt.subplots(rows, cols, figsize=(cols * 10, rows * 10))
        axes = axes.flatten()  # Flatten for easy iteration

        for i, (image, true_label, pred_label) in enumerate(failed_predictions):
            ax = axes[i]
            ax.imshow(image)  # Assuming images are PIL images
            ax.set_title(f"True: {labels[true_label]}\nPred: {labels[pred_label]}")
            ax.axis("off")

        # Hide extra subplots if any
        for j in range(i + 1, len(axes)):
            axes[j].axis("off")

        plt.tight_layout()
        plt.savefig(os.path.join(output_dir, "failed_predictions.png"))
        plt.close()

    # Save confusion matrix plot
    def save_cm_plot(matrix, title, filename, fmt="d"):
        plt.figure(figsize=(10, 8))
        sns.heatmap(matrix, annot=True, fmt=fmt, cmap="Blues", xticklabels=labels, yticklabels=labels)
        plt.xlabel("Predicted Label")
        plt.ylabel("True Label")
        plt.title(title, pad=20)
        plt.xticks(rotation=30)
        plt.yticks(rotation=30)
        plt.savefig(os.path.join(output_dir, filename), bbox_inches="tight", pad_inches=0.3)
        plt.close()

    save_cm_plot(cm, f"{model_safe_name} Confusion Matrix", f"confusion_matrix.png")
    save_cm_plot(cm_normalized, f"{model_safe_name} Confusion Matrix", f"normalized_confusion_matrix.png", fmt=".2f")


In [ ]:
def save_dataset_info(dataset, output_dir):
    info_path = os.path.join(output_dir, "dataset_info.json")
    # Get class distribution
    labels = dataset.features["label"].names
    label_counts = {label: 0 for label in labels}

    for example in dataset:
        label_counts[labels[example["label"]]] += 1

    dataset_info = {
        "num_samples": len(dataset),
        "num_classes": len(labels),
        "class_distribution": label_counts,
    }

    # Save dataset info as JSON
    with open(info_path, "w") as f:
        json.dump(dataset_info, f, indent=4)

    # Save class distribution plot
    plt.figure(figsize=(10, 6))
    sns.barplot(x=list(label_counts.keys()), y=list(label_counts.values()), palette="viridis")
    plt.xticks(rotation=30)
    plt.xlabel("Classes")
    plt.ylabel("Count")
    plt.title("Class Distribution in Test Dataset")
    plt.savefig(os.path.join(output_dir, "class_distribution.png"), bbox_inches="tight", pad_inches=0.3)
    plt.close()

    print("✅ Dataset info saved.")

## Main Function
This function loads the dataset, evaluates models, and saves reports to Google Drive.

In [ ]:

def main():
    """Mounts Google Drive, loads dataset, evaluates models, and saves reports."""
    drive.mount("/content/drive")

    models_path = [
        "SodaXII/resnet-152_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/convnextv2-base-1k-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/vit-hybrid-base-bit-384_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/vit-base-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/swin-base-patch4-window7-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/deit-base-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/dinov2-base_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/mobilevit-small_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/efficientnet-b0_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/convnextv2-femto-1k-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/vit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/swin-tiny-patch4-window7-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/deit-tiny-patch16-224_rice-leaf-disease-augmented-v4_fft",
        "SodaXII/resnet-152_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/convnextv2-base-1k-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/vit-hybrid-base-bit-384_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/vit-base-patch16-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/swin-base-patch4-window7-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/deit-base-patch16-224_rice-leaf-disease-augmented-v4_tl",
        "SodaXII/dinov2-base_rice-leaf-disease-augmented-v4_tl"
    ]


    dataset = load_dataset("cvmil/rice-leaf-disease-augmented-v4", split="test")
    labels = dataset.features["label"].names

    output_dir = "/content/drive/Shareddrives/CS198-Drones/[v4] Model Evaluation/"
    os.makedirs(output_dir, exist_ok=True)

    save_dataset_info(dataset, output_dir)

    for model_name in models_path:
        try:
            y_true, y_pred, failed_predictions, elapsed_time = evaluate_model(model_name, dataset, labels)
            generate_report(y_true, y_pred, labels, model_name, output_dir, failed_predictions, elapsed_time)
        except Exception as e:
            print(f"⚠️⚠️⚠️ Error processing {model_name}: {e}")

    print("✅ Evaluation completed. Reports saved to Google Drive.")

if __name__ == "__main__":
    main()
